The very first step is to download and install the FETPAPI wheel (which depends on FESAPI) : pip install fetpapi

Once FETPAPI wheel installed, you need to first import FESAPI (required dependency) and then fetpapi in order to be able to call its API.

In [ ]:
import fesapi
import fetpapi

Let's now connect to an ETP server by defining some few initialization parameters such as the URL and the authorization

In [ ]:
import uuid

etp_server_url = "wss://.../"
authorization = "Bearer ..."
initialization_params = fetpapi.InitializationParameters(str(uuid.uuid4()), etp_server_url)

Some ETP servers such as the OSDU RDDMS one require some extra header fields. Let's add them

In [ ]:
additionalHeaderField = fetpapi.MapStringString()
additionalHeaderField["data-partition-id"] = "osdu"
initialization_params.setAdditionalHandshakeHeaderFields(additionalHeaderField)

We can now create the ETP session.

In [ ]:
client_session = fetpapi.createClientSession(initialization_params, authorization)

And we can finally run the session in parallel not to block the main thread

In [ ]:
def start_etp_server(client_session):
    client_session.run()

from threading import Thread
t = Thread(target=start_etp_server, args=(client_session,), daemon=True)
t.start()

Let's wait for the connection to be effective

In [ ]:
from time import sleep, perf_counter
start_time = perf_counter()
while client_session.isEtpSessionClosed() and perf_counter() - start_time < 5:
    sleep(0.25)	
if client_session.isEtpSessionClosed():
    print("The ETP session could not be established in 5 seconds.")
else:
    print("Now connected to ETP Server")

Let's list the available dataspaces

In [ ]:
all_dataspaces = client_session.getDataspaces()
if all_dataspaces.empty() :
    print("There is no dataspace on this ETP server.")
for dataspace in all_dataspaces:
    print(dataspace.uri)

Let's list the content of the first dataspace

In [ ]:
dataspace = all_dataspaces.front()
etp_context = fetpapi.ContextInfo()
etp_context.uri = dataspace.uri
etp_context.depth = 1
all_resources = client_session.getResources(etp_context, fetpapi.ContextScopeKind__self);
if all_resources.empty() :
    print("There is no resource in dataspace " + dataspace.uri)
for resource in all_resources:
    print(resource.uri)

We can now close the session

In [ ]:
client_session.close()